# Analiza elekričnih avtomobilov

Najprej bom predstavil enote za posamezni podatek:

- doseg ... *[km]* ... količina kilometrov, ki jih avtomobil prepelje s polno baterijo,
- kapaciteta baterije ... *[kWh]* ... uporabna kapaciteta baterije,
- ucinkovitost ...*[Wh/km]* ... učinkovitost pri standardnih pogojih,
- hitro polnenje ... *[kW]* ... povprečna polnilna moč,
- teza ... *[kg]* ... teža avtomobila,
- vleka ... *[kg]* ... težaj ki jo lahko vleče avto,
- pospesek ... *[s]* ... čas, ki ga porabi avtomobil, da doseže hitrost 100 km/h pri začetni hitrosti 0,
- prtljaga ... *[L]* ... volumen prtljažnika,
- en postanek ... *[km]* ... skupni doseg pri avtocestni hitrosti z enim 15-minutnim postankom za polnjenje,
- price range ... *[€/km]* ... cena na kilometer dosega: pokazatelj vrednosti za denar. Kombinacija dostopnosti in dosega, nižja številka je boljša,
- cena ... *[€]* ... cena avtomobila.

Vse podatke sem zajel iz spletne strani [EV avtomobili](https://ev-database.org/#group=vehicle-group&rs-pr=10000_100000&rs-er=0_1000&rs-ld=0_1000&rs-ac=2_23&rs-dcfc=0_400&rs-ub=10_200&rs-tw=0_2500&rs-ef=100_350&rs-sa=-1_5&rs-w=1000_3500&rs-c=0_5000&rs-y=2010_2030&s=1&p=0-50)

## Analiza podatkov

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

- Najprej si poglejmo razpredelnico vseh avtomobilov.

In [ ]:
avtomobili = pd.read_csv("avtomobili.csv", index_col="id")

In [ ]:
avtomobili

In [ ]:
avtomobili.iloc[4]

- Prva stvar, ki me je zanimala je katerih 10 avtomobilov ima največ dosega.

In [ ]:
avtomobili[["naslov", "doseg"]].sort_values("doseg", ascending=False).head(10)

- Vidimo da so med najboljšimi 5-imi kar 4 avtomobili znamke Lucid. Presenetil pa me je podatek, da imajo avtomobili znamke Mercedes Benz tako veliko dosega. Pričakoval sem tudi kakšno Teslo na razpredelnici, ki pa je ni:

- Pa poglejmo, koliko avtomobilov ima dosega za več kot 500 km.

In [ ]:
sum(avtomobili.doseg > 500)

In [ ]:
avtomobili[avtomobili.doseg > 500]

- Ali je doseg avtomobila večji, če je le ta dražji?

In [ ]:
povprecni_doseg = avtomobili.groupby("cena")["doseg"].mean()


plt.figure(figsize=(9,5))
povprecni_doseg.plot(
    color="teal",        
    linewidth=2         
)

plt.title("Povprečni doseg glede na ceno avtomobila", fontsize=14, fontweight="bold", pad=10)
plt.xlabel("Cena (v EUR)", fontsize=12)
plt.ylabel("Povprečni doseg (v km)", fontsize=12)

plt.grid(True, linestyle="--", alpha=0.6)




- Kot pričakovano vidimo, da doseg s ceno avtomobila narašča.

- Ali imajo avtomobili s večjo kapaciteto baterije res več dosega?

In [ ]:
povprecni_doseg = avtomobili.groupby("kapaciteta_baterije")["doseg"].mean()

plt.figure(figsize=(9,5))
povprecni_doseg.plot(
    color="darkorange",  
    linewidth=2,          
    marker="o",           
    markersize=4,
    alpha=0.9             
)

plt.title("Povprečni doseg glede na kapaciteto baterije", fontsize=14, fontweight="bold", pad=10)
plt.xlabel("Kapaciteta baterije (kWh)", fontsize=12)
plt.ylabel("Povprečni doseg (v km)", fontsize=12)

plt.grid(True, linestyle="--", alpha=0.6)


- Da, seveda. 

- Ponavadi voznikov električnih avtomobilov odvetujejo vleko prikolice. Vidim pa, da ni pri vseh avtomobilih tako.

In [ ]:
avtomobili[["naslov", "teza", "vleka"]].sort_values("vleka")

- Kateri pa so avti, ki ne morejo vleči prikolice?

In [ ]:
avtomobili[avtomobili.vleka == 0]

In [ ]:
sum(avtomobili.vleka == 0)

In [ ]:
sum(avtomobili.vleka == 0) / 1140 * 100 

- Tretjina električnih avtomobilov ne more vleči prikolice, ker jim to posledično uničuje elektromotorje.

Poglejmo graf, ki prikazuje koliko lahko avtomobili vlečejo glede na kapaciteto baterije.

In [ ]:
povprecna_vleka = avtomobili.groupby("kapaciteta_baterije")["vleka"].mean()

plt.figure(figsize=(9,5))
povprecna_vleka.plot(
    color="royalblue",    
    linewidth=2.5,               
    linestyle="-",        
    alpha=0.9             
)

plt.title("Povprečna vleka glede na kapaciteto baterije", fontsize=14, fontweight="bold", pad=10)
plt.xlabel("Kapaciteta baterije (kWh)", fontsize=12)
plt.ylabel("Povprečna vleka (v Nm)", fontsize=12)

plt.grid(True, linestyle="--", alpha=0.6)

- Tudi avtomobili z večjo kapaciteto baterije ne smejo vleči prikolice.

- Če primerjamo pospešek 0-100 *km*/*h* električnega avtomobila in ne električnega bi moral biti pri električnem manjši zaradi stalne moči elektromotorja. Zanima me, katerih 20 avtomobilov najhitreje pospešuje.

In [ ]:
avtomobili[["naslov", "pospesek"]].sort_values("pospesek", ascending=True).head(20)

- Ponovno je na prvem mestu avtomobil znamke Lucid. Vidim pa da je v razpredelnici dosti Porsche-jev. Koliko pa je povprečje?

In [ ]:
float(avtomobili.pospesek.mean())

- Rekel bi, da je to kar hitro za povprečje

In [ ]:
avtomobili["pospesek_zaokrozen"] = avtomobili["pospesek"].round(1)

povp_cena = (
    avtomobili
    .groupby("pospesek_zaokrozen")
    .cena
    .mean()
    .sort_index()
)

fig, ax = plt.subplots(figsize=(14, 7))  

povp_cena.plot(
    kind="bar",
    ax=ax,
    edgecolor="black"
)

ax.set_title("Povprečna cena glede na zaokrožen pospešek", fontsize=14, fontweight="bold", pad=10)
ax.set_xlabel("Pospešek (s od 0 do 100 km/h)")
ax.set_ylabel("Cena (EUR)")

plt.xticks(
    ticks=range(len(povp_cena.index)),
    labels=[f"{x:.1f}" for x in povp_cena.index],
    rotation=60,
    ha="right",
    fontsize=9
)




- Vidimo, da so dražji avtomobili hitrejši, razen določenih izjem.

- Pa poglejmo kakšen volumen prtljaznika imajo avtomobili.

In [ ]:
avtomobili[["naslov", "teza", "prtljaga"]]

In [ ]:
scatter = avtomobili.plot.scatter(
    x="teza",
    y="prtljaga",
    figsize=(8, 6),
    s=70,              
    alpha=0.7,         
    edgecolor="black"  
)

scatter.set_title("Razmerje med težo vozila in velikostjo prtljažnika", fontsize=14, fontweight="bold", pad=10)
scatter.set_xlabel("Teža vozila (kg)")
scatter.set_ylabel("Prostornina prtljažnika (L)")

scatter.grid(True, linestyle="--", linewidth=0.5)

- Vidimo, da se večina giba v povprečju oz. skupaj, le malo je odstopanj

- Sedaj pa recimo, da želim kupiti avto, ki bo od 0-100 *km*/*h* pospešil v manj kot 5 *s*, in bom z njim naredil več kot 600 *km*. Seveda pa nesme bit predrag, torej manj kot 30.000 *€*

In [ ]:
avtomobili[(avtomobili.pospesek < 5) & (avtomobili.doseg > 600) & (avtomobili.cena < 30000)]

- Aha, vidim da tak avto ne obstaja. Ti šment. Priredimo malo kriterije.

In [ ]:
avtomobili[(avtomobili.pospesek < 6.5) & (avtomobili.doseg > 300) & (avtomobili.cena < 40000)]


- Moram priznati, da sem pričakoval več avtomobilov, ki bo ustrezalo danim kriterijem.

- Za konec pa si poglejmo še primerjavo s hitrim polnenjem

In [ ]:
avtomobili[["naslov", "kapaciteta_baterije", "hitro_polnenje"]]

In [ ]:
avtomobili[["naslov", "kapaciteta_baterije", "hitro_polnenje"]].sort_values("hitro_polnenje", ascending=False)

- Vidimo, da znamak XPENG prevladuje v tem razdelku.

In [ ]:
povp_hitro = (
    avtomobili
    .groupby("kapaciteta_baterije")
    .hitro_polnenje
    .mean()
)

graf = povp_hitro.plot(
    kind="line",
    marker="o",
    linewidth=2,
    figsize=(9, 5)
)

graf.set_title("Povprečno hitro polnjenje glede na kapaciteto baterije", fontsize=14, fontweight="bold", pad=10)
graf.set_xlabel("Kapaciteta baterije (kWh)")
graf.set_ylabel("Hitro polnjenje (kW)")

graf.grid(True, linestyle="--", linewidth=0.5)

- Vidimo, da ima manjša kapaciteta baterije tudi manjšo polnilno moč

## Zaključek

Skozi to analizo smo opazili:
- Da ima znamka Lucid avte z največ dosega,
- Da imajo dražji avti nasploh večji doseg
- Da tretjina avtomobilov ne sme vleči prikolice
- Da najbolši avto v pospeških potebuje 2 s od 0-100 km/h
- Da imajo avti znamke XPENG največjo polnilno moč